In [209]:
import matplotlib as plt
import numpy as np 
import queue as Q 
import heapq as hq 
from decimal import Decimal

In [210]:
precision = 2

In [211]:
class Queue:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return len(self.items) == 0

    def enqueue(self, item):
        self.items.append(item)

    def dequeue(self):
        if(self.isEmpty()):
            print("unable to dequeue")
            return None
        node = self.items[0]
        del self.items[0]
        return node
    def prioritySort(self):
        self.items = sorted(self.items, key=lambda process:(-process.priority, process.processNumber))
    def size(self):
        return len(self.items)
    
    def front(self):
        if(self.isEmpty()):
            return None
        return self.items[0]
    def printQueue(self):
        for item in self.items:
            item.printProcess()
    def calculateStats(self,outputFile):
        averageTAT = 0 
        weightedAverageTAT = 0
        f = open(outputFile,'w')
        for item in self.items:
            item.writeStats(f)
            averageTAT += item.TAT()
            weightedAverageTAT += item.weightedTAT()
        averageTAT /= self.size()
        weightedAverageTAT /= self.size()
        f.write("\naverage TAT = " + str(np.round(averageTAT, precision)) + "\nweightedAverageTAT = " \
                + str(np.round(weightedAverageTAT, precision)))
        f.close()
        

In [212]:
class MyHeap(object):
    def __init__(self,initial=None, key=lambda x:x):
        self.key = key
        if initial:
            self._data = [(key(item), item) for item in initial]
            hq.heapify(self._data)
        else:
            self._data = []

    def push(self, item):
        hq.heappush(self._data, (self.key(item), item))
        
    def pop(self):
        return hq.heappop(self._data)[1]
    def top(self):
        return self._data[0][1]
    def heapPushPop(self,item):
        hq.heappush(self._data,(self.key(item), item))
        return hq.heappop(self._data)[1]
    def size(self):
        return len(self._data)

In [213]:
def getTimeLeft(process):
    return np.round(process.timeLeft, precision)

class Process:
    def __init__(self, processNumber, arrivalTime, burstTime, priority):
        self.processNumber = processNumber
        self.arrivalTime = arrivalTime
        self.burstTime = burstTime
        self.priority = priority
        self.timeLeft = burstTime
        self.transitions = []

    def __lt__(self, other):
        if self.priority == other.priority:
            return self.processNumber < other.processNumber
        return self.priority > other.priority

    def getTransitions(self):
        for i in range(len(self.transitions)):
            print("Transition # " + str(i) + " from " + str(self.transitions[i][0]) + " to " + str(
                self.transitions[i][1]) + "\n")
        return

    def setTransitions(self, start, end):
        self.transitions.append((start, end))
        return

    def setTimeLeft(self, quantum):
        self.timeLeft -= quantum
        np.round(self.timeLeft, precision)
        return

    def getTimeLeft(self):
        return np.round(self.timeLeft, precision)

    def endTime(self, endTime):
        self.endTime = endTime
        return

    def startTime(self, startTime):
        self.startTime = startTime
        return
    
    def waitTime(self):
        if(self.transitions == 0):
            return np.round((self.startTime - self.arrivalTime), precision)
        else:
            self.startTime = self.transitions[0][0] ## first element in the first tuple is the startTime
            return np.round((self.startTime - self.arrivalTime), percision)
    def TAT(self):
        if(self.transitions ==0):
            return np.round((self.endTime - self.arrivalTime), precision)
        else:
            self.endTime = self.transitions[-1][1] ## second element in the last tuple is the endTime
            return np.round((self.endTime - self.arrivalTime),percision)
   
    def weightedTAT(self):
        return np.round((self.endTime - self.arrivalTime) / self.burstTime, precision)

    def printProcess(self):
        print("PNumber: " + str(self.processNumber) + ", Arrival: " + str(self.arrivalTime) + ", Burst: " \
              + str(self.burstTime) + ", Priority: " + str(self.priority))

    def writeStats(self, f):
        f.write("Process #" + str(self.processNumber) + " wait time = " + str(self.waitTime()) + \
                " TAT = " + str(self.TAT()) + " weighted TAT = " + str(self.weightedTAT()) + " arrived at: " + \
                str(self.arrivalTime) + " started at: " + str(np.round(self.startTime, precision)) + " ended at: " \
                + str(np.round(self.endTime, precision)) + '\n')


In [214]:
## Take input from the file in an array of process
inputFile = open("testRR.txt",'r')
processArray = []

for line in inputFile:
    line = line.split(" ")
    if len(line) == 1 : 
        continue
    temp = Process(int(line[0]),float(line[1]),float(line[2]),int(line[3]))
    processArray.append(temp)

inputFile.close()

In [215]:
## sort the process according their arrival time
processArray = sorted(processArray, key=lambda process:float(process.arrivalTime))

for i in processArray:
    print(str(i.processNumber) + " " + str(i.arrivalTime) + " " + str(i.burstTime) + " " + str(i.priority) )
    


1 1.0 6.0 5
2 1.0 1.0 3
3 3.0 7.0 3
4 4.0 5.0 1
5 5.0 2.0 3


In [216]:
'''
myHeap = MyHeap()
for i in processArray:
    myHeap.push((i.timeLeft, i))
    
currentProcess = myHeap.pop()
print(currentProcess.printProcess())        
'''

'\nmyHeap = MyHeap()\nfor i in processArray:\n    myHeap.push((i.timeLeft, i))\n    \ncurrentProcess = myHeap.pop()\nprint(currentProcess.printProcess())        \n'

In [217]:
myHeap = MyHeap(key = getTimeLeft)
outqueue = Queue()
exit = False
time = 0.0 
size = len(processArray)
currentProcess = None
cpuCycle = 1
switchingTime = 0.0
while(True):
    
    if(exit or (len(processArray) == 0 and outqueue.size() == size)): ## will be changed 
        break
        
    while(len(processArray) > 0 and time >= float(processArray[0].arrivalTime)):
        myHeap.push(processArray[0])
        del processArray[0]
    if(currentProcess != None):
        if (currentProcess.getTimeLeft() <= 0):
            print("removing ended process and waiting for arrival of new process / switching process") 
            end = time + currentProcess.getTimeLeft() 
            currentProcess.setTransitions(start,end)
            outqueue.enqueue(currentProcess)
            currentProcess = None 
            
        if(currentProcess != None):
            currentProcess.setTimeLeft(cpuCycle)  
            
    if(myHeap.size() > 0):
        if(currentProcess != None):
            temp = myHeap.top()
            if(temp.getTimeLeft() < currentProcess.getTimeLeft()):
                end = time
                currentProcess.setTransitions(start,end)
                time += switchingTime
                currentProcess = myHeap.heapPushPop(currentProcess)
                currentProcess.setTimeLeft(cpuCycle)
                start = time
        else:
            
            print("putting a new process")
            currentProcess = myHeap.pop()
            print(currentProcess.printProcess())
            time += switchingTime
            start = time
            currentProcess.setTimeLeft(cpuCycle)

        
     
    print(time)
    time += cpuCycle
    time = np.round(time,5)

0.0
putting a new process
PNumber: 2, Arrival: 1.0, Burst: 1.0, Priority: 3
None
1.0
removing ended process and waiting for arrival of new process / switching process
putting a new process
PNumber: 1, Arrival: 1.0, Burst: 6.0, Priority: 5
None
2.0
3.0
4.0
5.0
6.0
7.0
removing ended process and waiting for arrival of new process / switching process
putting a new process
PNumber: 5, Arrival: 5.0, Burst: 2.0, Priority: 3
None
8.0
9.0
removing ended process and waiting for arrival of new process / switching process
putting a new process
PNumber: 4, Arrival: 4.0, Burst: 5.0, Priority: 1
None
10.0
11.0
12.0
13.0
14.0
removing ended process and waiting for arrival of new process / switching process
putting a new process
PNumber: 3, Arrival: 3.0, Burst: 7.0, Priority: 3
None
15.0
16.0
17.0
18.0
19.0
20.0
21.0
removing ended process and waiting for arrival of new process / switching process
22.0


In [218]:
for i in range(outqueue.size()):
    rightnow = outqueue.dequeue()
    print("Process #  " + str(rightnow.processNumber) + "\n" )
    rightnow.getTransitions()

Process #  2

Transition # 0 from 1.0 to 2.0

Process #  1

Transition # 0 from 2.0 to 8.0

Process #  5

Transition # 0 from 8.0 to 10.0

Process #  4

Transition # 0 from 10.0 to 15.0

Process #  3

Transition # 0 from 15.0 to 22.0



In [219]:
heapheap = MyHeap(key = getTimeLeft)
process1 = Process(1,1.0,6,5)
process2 = Process(2,2.0,1,3)
heapheap.push(process1)
heapheap.push(process2)
heapheap.top().printProcess()

PNumber: 2, Arrival: 2.0, Burst: 1, Priority: 3
